In [ ]:
!pip install scikit-plot

In [ ]:
# Load libraries
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn import preprocessing
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, auc
from sklearn.metrics import plot_confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import learning_curve
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import scikitplot as skplt
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_curve, auc
from scipy import interp
from itertools import cycle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
 # data is being refined, Here we fist get our data to be converted into pandas dataframae
# then we drop empty spaces and eradicate useless indices and getting data in float type
def cleaningdata(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)
# fetching dataset from path
dataset = pd.read_csv('/content/drive/MyDrive/Code/dataset/IoT-BoT.csv', encoding='utf-8')
# for object type data we will label it and transform into apporopriate type fo data after using fit_transform on that colomn to avoid errors
for column in dataset.columns:
    if dataset[column].dtype == type(object):
        le = LabelEncoder()
        dataset[column] = le.fit_transform(dataset[column])
ds = cleaningdata(dataset)
X = ds.iloc[:, 0:85].values
# collecting our LABEL colomn in y
y = ds.iloc[:, -2].values
#normalize
sc = StandardScaler()
X = sc.fit_transform(X)
y = label_binarize(y, classes=[0, 1, 2, 3, 4])
n_classes = y.shape[1]
n_classes
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)


In [ ]:
target_names = ['DoS', 'DDOS', 'Reconnaissance', 'Normal','Theft']

In [ ]:
#decision tree with 5 fold cv
from sklearn import tree
pipe_tree = Pipeline([('clf', OneVsRestClassifier(tree.DecisionTreeClassifier(max_depth=3)))])
train_sizes_tree, train_scores_tree, test_scores_tree = learning_curve(estimator=pipe_tree,X=X, y=y, train_sizes=np.linspace(0.2,1.0,5), cv=5, n_jobs=-1)
train_mean_tree = np.mean(train_scores_tree, axis=1)
train_std_tree = np.std(train_scores_tree, axis=1)
test_mean_tree = np.mean(test_scores_tree, axis=1)
test_std_tree = np.std(test_scores_tree, axis=1)

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [ ]:
for i in train_mean_tree:
    print(i)

0.997074682247747
0.9970927198819824
0.9970772590526377
0.9970999671457376
0.9971053480906853


In [ ]:
for i in test_mean_tree:
    print(i)

0.9971016121177525
0.9971062503605801
0.9971047042796377
0.9971052196399517
0.9971052196399517


In [ ]:
#dt training
pipe_tree = pipe_tree.fit(X_train, y_train)
y_pred_test = pipe_tree.predict(X_test)
y_pred_train=pipe_tree.predict(X_train)
# print dt_test_scores of accuracy,precision,recall,f1-score
print("Test_Accuracy:",metrics.accuracy_score(y_test, y_pred_test))
print("Test_Precision:",metrics.precision_score(y_test, y_pred_test, average='weighted'))
print("Test_Recall:",metrics.recall_score(y_test, y_pred_test, average='weighted'))
print("Test_f1-score:",metrics.f1_score(y_test, y_pred_test, average='weighted'))
print(classification_report(y_test, y_pred_test, target_names=target_names))

# print dt_train_scores of accuracy,precision,recall,f1-score
print("Train_Accuracy:",metrics.accuracy_score(y_train, y_pred_train))
print("Train_Precision:",metrics.precision_score(y_train, y_pred_train, average='weighted'))
print("Train_Recall:",metrics.recall_score(y_train, y_pred_train, average='weighted'))
print("Train_f1-score:",metrics.f1_score(y_train, y_pred_train, average='weighted'))
#dt classification report
print(classification_report(y_train, y_pred_train, target_names=target_names))

Test_Accuracy: 0.9971165590422544
Test_Precision: 0.9971436175563861
Test_Recall: 0.9999974231984292
Test_f1-score: 0.9985644315638786


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                precision    recall  f1-score   support

           DoS       1.00      1.00      1.00    126985
          DDOS       0.99      1.00      1.00    130325
Reconnaissance       1.00      1.00      1.00     19453
        Normal       1.00      1.00      1.00    111198
         Theft       1.00      1.00      1.00       117

     micro avg       1.00      1.00      1.00    388078
     macro avg       1.00      1.00      1.00    388078
  weighted avg       1.00      1.00      1.00    388078
   samples avg       1.00      1.00      1.00    388078

Train_Accuracy: 0.99710238476697
Train_Precision: 0.9971322186157606
Train_Recall: 0.9999948463935384
Train_f1-score: 0.9985573999207358
                precision    recall  f1-score   support

           DoS       1.00      1.00      1.00    509554
          DDOS       0.99      1.00      1.00    520797
Reconnaissance       1.00      1.00      1.00     77744
        Normal       1.00      1.00      1.00    443813
         Theft     

In [ ]:
#random forest
from sklearn.ensemble import RandomForestClassifier
pipe_rnd = Pipeline([('clf', OneVsRestClassifier(RandomForestClassifier(max_depth=3, n_estimators=100)))])
train_sizes_rnd, train_scores_rnd, test_scores_rnd = learning_curve(estimator=pipe_rnd,X=X, y=y, train_sizes=np.linspace(0.2,1.0,5), cv=5, n_jobs=-1)
train_mean_rnd = np.mean(train_scores_rnd, axis=1)
train_std_rnd = np.std(train_scores_rnd, axis=1)
test_mean_rnd = np.mean(test_scores_rnd, axis=1)
test_std_rnd = np.std(test_scores_rnd, axis=1) 

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [ ]:
for i in train_mean_tree:
    print(i)

0.997074682247747
0.9970927198819824
0.9970772590526377
0.9970999671457376
0.9971053480906853


In [ ]:
for i in test_mean_tree:
    print(i)

0.9971016121177525
0.9971062503605801
0.9971047042796377
0.9971052196399517
0.9971052196399517


In [ ]:
#rf training
pipe_rnd = pipe_rnd.fit(X_train, y_train)
y_pred_test = pipe_rnd.predict(X_test)
y_pred_train=pipe_rnd.predict(X_train)
# print rf_test_scores of accuracy,precision,recall,f1-score
print("Test_Accuracy:",metrics.accuracy_score(y_test, y_pred_test))
print("Test_Precision:",metrics.precision_score(y_test, y_pred_test, average='weighted'))
print("Test_Recall:",metrics.recall_score(y_test, y_pred_test, average='weighted'))
print("Test_f1-score:",metrics.f1_score(y_test, y_pred_test, average='weighted'))
print(classification_report(y_test, y_pred_test, target_names=target_names))


# print rf_train_scores of accuracy,precision,recall,f1-score
print("Train_Accuracy:",metrics.accuracy_score(y_train, y_pred_train))
print("Train_Precision:",metrics.precision_score(y_train, y_pred_train, average='weighted'))
print("Train_Recall:",metrics.recall_score(y_train, y_pred_train, average='weighted'))
print("Train_f1-score:",metrics.f1_score(y_train, y_pred_train, average='weighted'))
#rf classification report
print(classification_report(y_train, y_pred_train, target_names=target_names))


Test_Accuracy: 0.9804961889104767
Test_Precision: 0.9945947412248063
Test_Recall: 0.9819134297744267
Test_f1-score: 0.9881874961794918


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                precision    recall  f1-score   support

           DoS       1.00      0.98      0.99    126996
          DDOS       0.98      0.97      0.98    130265
Reconnaissance       1.00      1.00      1.00     19531
        Normal       1.00      1.00      1.00    111185
         Theft       1.00      0.72      0.84       101

     micro avg       0.99      0.98      0.99    388078
     macro avg       1.00      0.93      0.96    388078
  weighted avg       0.99      0.98      0.99    388078
   samples avg       0.98      0.98      0.98    388078

Train_Accuracy: 0.980472340916221
Train_Precision: 0.9947084488224693
Train_Recall: 0.9818451328374276
Train_f1-score: 0.9882089356359396
                precision    recall  f1-score   support

           DoS       1.00      0.98      0.99    509543
          DDOS       0.98      0.97      0.98    520857
Reconnaissance       1.00      1.00      1.00     77666
        Normal       1.00      1.00      1.00    443826
         Theft    

In [ ]:
# svm
from sklearn.svm import SVC
pipe_svc = Pipeline([('clf', OneVsRestClassifier(SVC(C=1000)))])
train_sizes_svc, train_scores_svc, test_scores_svc = learning_curve(estimator=pipe_svc,X=X, y=y, train_sizes=np.linspace(0.2,1.0,5), cv=5, n_jobs=-1)
train_mean_svc = np.mean(train_scores_svc, axis=1)
train_std_svc = np.std(train_scores_svc, axis=1)
test_mean_svc = np.mean(test_scores_svc, axis=1)
test_std_svc = np.std(test_scores_svc, axis=1)


In [ ]:
for i in train_mean_svc:

    print(i)

0.9979813583285333
0.9987196941928052
0.9981358328533277
0.9983308896685343
0.9986102529193067


In [ ]:
for i in test_mean_svc:
    print(i)

0.8594590621075193
0.9104634836128982
0.8479077108132428
0.8817434017691494
0.8273432344229221


In [ ]:
#svm training
pipe_svc = pipe_svc.fit(X_train, y_train)
y_pred_test = pipe_svc.predict(X_test)
y_pred_train=pipe_svc.predict(X_train)
# print svc_test_scores of accuracy,precision,recall,f1-score
print("Test_Accuracy:",metrics.accuracy_score(y_test, y_pred_test))
print("Test_Precision:",metrics.precision_score(y_test, y_pred_test, average='weighted'))
print("Test_Recall:",metrics.recall_score(y_test, y_pred_test, average='weighted'))
print("Test_f1-score:",metrics.f1_score(y_test, y_pred_test, average='weighted'))
print(classification_report(y_test, y_pred_test, target_names=target_names))


# print svc_train_scores of accuracy,precision,recall,f1-score, cohens kappa
print("Train_Accuracy:",metrics.accuracy_score(y_train, y_pred_train))
print("Train_Precision:",metrics.precision_score(y_train, y_pred_train, average='weighted'))
print("Train_Recall:",metrics.recall_score(y_train, y_pred_train, average='weighted'))
print("Train_f1-score:",metrics.f1_score(y_train, y_pred_train, average='weighted'))
#svm classification report
print(classification_report(y_train, y_pred_train, target_names=target_names))